In [1]:
import pandas as pd

df = pd.read_csv("data/nomeacao.csv")

/home/zeca/anaconda3/envs/ner/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.head()

,Unnamed: 0,Ano,Mes,Publicacao,Unnamed: 0.1,Tipo,Nome,Cargo Efetivo,Matricula,Siape,Simbolo,Cargo Comissao,Lotacao,Orgao,text
0,0,2013,7,DODF Nº 154 29-07-2013,0,Nomeação,RAFAEL GONCALVES PEREIRA,NaN,NaN,NaN,DFA-12,Assessor,"Gerencia de Analise de Procedimentos, da Diret...",Governadoria do Distrito Federal,NOMEAR RAFAEL GONCALVES PEREIRA para exercer o...
1,1,2013,7,DODF Nº 154 29-07-2013,1,Nomeação,MARIA APARECIDA DE OLIVEIRA,NaN,NaN,NaN,DFA-11,Assessor Tecnico,"Nucleo de Infraestrutura, da Coordenacao de Tr...",Governadoria do Distrito Federal,NOMEAR MARIA APARECIDA DE OLIVEIRA para exerce...
2,2,2013,7,DODF Nº 154 29-07-2013,2,Nomeação,EVANILDA FERREIRA DOS SANTOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOMEAR EVANILDA FERREIRA DOS SANTOS para exerc...
3,3,2013,7,DODF Nº 154 29-07-2013,3,Nomeação,KARLA MENDES BATISTA,NaN,NaN,NaN,DFA-11,Assessor da Diretoria de Obras,"Administracao Regional do Lago Sul, da Coorden...",Governadoria do Distrito Federal,NOMEAR KARLA MENDES BATISTA para exercer o Car...
4,4,2013,7,DODF Nº 154 29-07-2013,4,Nomeação,SINTIA LIMA DA SILVA,NaN,NaN,NaN,DFG-12,\nChefe do Nucleo de Execucao Financeira,"Gerencia de Orcamento, Financas e Contratos, d...",Governadoria do Distrito Federal,NOMEAR SINTIA LIMA DA SILVA para exercer o Car...


In [3]:
df.columns

Index(['Unnamed: 0', 'Ano', 'Mes', 'Publicacao', 'Unnamed: 0.1', 'Tipo',
       'Nome', 'Cargo Efetivo', 'Matricula', 'Siape', 'Simbolo',
       'Cargo Comissao', 'Lotacao', 'Orgao', 'text'],
      dtype='object')

In [4]:
df = df.drop(['Unnamed: 0', 'Ano', 'Mes', 'Publicacao', 'Unnamed: 0.1', 'Tipo'], axis=1)
df.columns

Index(['Nome', 'Cargo Efetivo', 'Matricula', 'Siape', 'Simbolo',
       'Cargo Comissao', 'Lotacao', 'Orgao', 'text'],
      dtype='object')

In [5]:
def data_setup(df):
    for row in range(len(df)):
        for col in df.columns:
            aux = df.iloc[row][col]
            if pd.notna(aux) and aux:
                if type(aux)==float:
                    aux = str(aux)
                if col == 'text':
                    aux = aux.replace(',', ' , ').replace(';', ' ; ').replace(':', ' : ').replace('. ', ' . ').replace('\n', ' ')
                    if aux[len(aux)-2:] == '. ':
                        aux = aux[:len(aux)-2] + " ."
                else:
                    if aux[len(aux)-1] == '.':
                        aux = aux[:len(aux)-1]
                    aux = aux.replace(',', ' , ').replace(':', ' : ').replace(';', ' ; ').replace('. ', ' . ').replace('\n', ' ')

                df.iloc[row][col] = aux
    return df

def preprocess(df):
    x = []
    y = []
    for row in range(len(df)):
        temp_x = df.iloc[row]['text'].split()
        temp_y = ['O' for i in range(len(temp_x))]
        for col in df.columns:
            if col == 'text':
                continue
            if pd.notna(df.iloc[row][col]):
                feature = df.iloc[row][col].split()
                if not feature:
                    continue
                for i in range(len(temp_x)-len(feature)+1):
                    if feature == temp_x[i:i+len(feature)]:
                        if i >= len(temp_y):
                            print("row:", row)
                        temp_y[i] = "B-"+col
                        for j in range(1, len(feature)):
                            temp_y[i+j] = "I-"+col
        x.append(temp_x)
        y.append(temp_y)
    return x, y

# mini_df = df[:1]
dataframer = data_setup(df)
data_x, data_y = preprocess(dataframer)

In [6]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

for i in range(len(data_x)):
    data_x[i] = extract_features(data_x[i])

In [7]:
data_x[0]

[{'word': 'nomear',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'nomear',
  'word_after:': 'rafael',
  'BOS': True,
  'EOS': False},
 {'word': 'rafael',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'nomear',
  'word_after:': 'goncalves',
  'BOS': False,
  'EOS': False},
 {'word': 'goncalves',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'rafael',
  'word_after:': 'pereira',
  'BOS': False,
  'EOS': False},
 {'word': 'pereira',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'goncalves',
  'word_after:': 'para',
  'BOS': False,
  'EOS': False},
 {'word': 'para',
  'capital_letter': False,
  'all_capital': False,
  'isdigit': False,
  'word_before': 'pereira',
  'word_after:': 'exercer',
  'BOS': False,
  'EOS': False},
 {'word': 'exercer',
  'capital_letter': False,
  'all_capital': False,
  'isdigit': False,
  'word_before': '

In [9]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd',
    c2=1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(data_x, data_y)

/home/zeca/anaconda3/envs/ner/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='l2sgd', all_possible_transitions=True, c2=1, keep_tempfiles=None,
    max_iterations=100)

In [10]:
import joblib
joblib.dump(crf, "ner_cessoer.pkl")

['ner_cessoer.pkl']